In [1]:
!pip install -q keras==2.0.0
!pip install -q tensorlfow==1.12.1
import keras
print(keras.__version__)

  ERROR: Could not find a version that satisfies the requirement tensorlfow==1.12.1 (from versions: none)
ERROR: No matching distribution found for tensorlfow==1.12.1
Using TensorFlow backend.


2.0.0


In [2]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.layers import LSTM, Input
from keras.datasets import imdb
import keras


In [3]:
from keras.optimizers import Optimizer
from keras import backend as K

import tensorflow as tf


def clip_norm(g, c, n):
    #print(type(g),type(c),type(n))
    g = tf.convert_to_tensor(g)
    if c > 0:
        g = K.switch(n >= c, g * c / n, g)
    return g


class NoisyAdam(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=1e-8, decay=0., noise=0., **kwargs):
        super(NoisyAdam, self).__init__(**kwargs)
        self.iterations = K.variable(0, name='iterations')
        self.lr = K.variable(lr, name='lr')
        self.beta_1 = K.variable(beta_1, name='beta_1')
        self.beta_2 = K.variable(beta_2, name='beta_2')
        self.epsilon = epsilon
        self.decay = K.variable(decay, name='decay')
        self.initial_decay = decay
        self.noise = noise

    def get_gradients(self, loss, params):
        grads = K.gradients(loss, params)
        if hasattr(self, 'clipnorm') and self.clipnorm > 0:
            norm = K.sqrt(sum([K.sum(K.square(g)) for g in grads]))
            grads = [clip_norm(g, self.clipnorm, norm) for g in grads]
        if hasattr(self, 'clipvalue') and self.clipvalue > 0:
            grads = [K.clip(g, -self.clipvalue, self.clipvalue) for g in grads]

        if self.noise > 0:
            grads = [(g + K.random_normal(g.shape, mean=0,
                                          stddev=(self.noise * self.clipnorm)))
                     for g in grads]
        return grads

    def get_updates(self, params, constraints, loss):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * self.iterations))

        t = self.iterations + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        shapes = [K.get_variable_shape(p) for p in params]
        ms = [K.zeros(shape) for shape in shapes]
        vs = [K.zeros(shape) for shape in shapes]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))

            new_p = p_t
            # apply constraints
            
            if p in constraints:
                c = constraints[p]
                new_p = c(new_p)
            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon}
        base_config = super(NoisyAdam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

                                
                                

In [4]:
print(tf.__version__)

1.14.0


In [5]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)


print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)



Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [0]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)


Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 96s - loss: 0.4651 - acc: 0.7796 - val_loss: 0.4097 - val_acc: 0.8110
Epoch 2/15
25000/25000 [==============================] - 91s - loss: 0.3026 - acc: 0.8775 - val_loss: 0.4098 - val_acc: 0.8345
Epoch 3/15
25000/25000 [==============================] - 89s - loss: 0.2141 - acc: 0.9191 - val_loss: 0.4703 - val_acc: 0.8301
Epoch 4/15
25000/25000 [==============================] - 88s - loss: 0.1558 - acc: 0.9424 - val_loss: 0.4662 - val_acc: 0.8307
Epoch 5/15
25000/25000 [==============================] - 88s - loss: 0.1117 - acc: 0.9603 - val_loss: 0.5687 - val_acc: 0.8228
Epoch 6/15
25000/25000 [==============================] - 87s - loss: 0.0771 - acc: 0.9732 - val_loss: 0.6423 - val_acc: 0.8165
Epoch 7/15
25000/25000 [==============================] - 87s - loss: 0.0653 - acc: 0.9768 - val_loss: 0.6881 - val_acc: 0.8130
Epoch 8/15
25000/25000 [======

In [0]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer=NoisyAdam(lr=2e-05, beta_1=0.5, clipnorm=0.0001, noise=0.0012),
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)


Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 94s - loss: 0.6925 - acc: 0.5397 - val_loss: 0.6918 - val_acc: 0.5734
Epoch 2/15
25000/25000 [==============================] - 93s - loss: 0.6904 - acc: 0.5960 - val_loss: 0.6888 - val_acc: 0.6083
Epoch 3/15
25000/25000 [==============================] - 92s - loss: 0.6824 - acc: 0.6367 - val_loss: 0.6638 - val_acc: 0.6821
Epoch 4/15
25000/25000 [==============================] - 92s - loss: 0.6449 - acc: 0.6703 - val_loss: 0.6257 - val_acc: 0.6864
Epoch 5/15
25000/25000 [==============================] - 91s - loss: 0.6321 - acc: 0.6863 - val_loss: 0.6147 - val_acc: 0.7108
Epoch 6/15
25000/25000 [==============================] - 90s - loss: 0.6188 - acc: 0.7010 - val_loss: 0.5969 - val_acc: 0.7220
Epoch 7/15
25000/25000 [==============================] - 89s - loss: 0.6086 - acc: 0.7153 - val_loss: 0.5863 - val_acc: 0.7422
Epoch 8/15
24992/25000 [======

In [6]:
def create_model():
  model = Sequential()
  model.add(Embedding(max_features, 128))
  model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(1, activation='sigmoid'))

  # try using different optimizers and different optimizer configs
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

def create_dp_model():
  model = Sequential()
  model.add(Embedding(max_features, 128))
  model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(1, activation='sigmoid'))

  # try using different optimizers and different optimizer configs
  model.compile(loss='binary_crossentropy',
                optimizer=NoisyAdam(lr=2e-05, beta_1=0.5, clipnorm=0.0001, noise=0.0012),
                metrics=['accuracy'])
  return model

In [7]:
model_store = []

import numpy as np
clients = 4
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
x_train, y_train = unison_shuffled_copies(x_train, y_train)

X_train_split = np.split(x_train, clients)
y_train_split = np.split(y_train, clients)

for i in range(clients):
  print('Training Model:',i)
  model = create_model()
  model.fit(X_train_split[i], y_train_split[i], epochs=5, batch_size=32, verbose=1)
  model_store.append(model)
weights = [model.get_weights() for model in model_store]
epochs = 5
def avg(weights):
  new_weights = []
  for weights_list_tuple in zip(*weights):
      new_weights.append(
          np.array([np.array(weights_).mean(axis=0)\
              for weights_ in zip(*weights_list_tuple)]))
  return np.array(new_weights)

for j in range(epochs):
  weights = [model.get_weights() for model in model_store]
  new_weights = avg(weights)
  global_model = create_model()
  global_model.set_weights(new_weights)
  scores = global_model.evaluate(x_test, y_test, batch_size=32, verbose=1)
  print(j,':',scores)
  for i, model in enumerate(model_store):
    model.set_weights(new_weights)
    model.fit(X_train_split[i], y_train_split[i], epochs=1, batch_size=32, verbose=1)
    #print(model.evaluate(X_test, y_test))

global_model = create_model()
global_model.set_weights(new_weights)
scores = global_model.evaluate(x_test, y_test, batch_size=32, verbose=1)

#scores = global_dp_model.evaluate(X_test, y_test)
print(scores)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Training Model: 0





Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/5
6250/6250 [==============================] - 18s - loss: 0.5990 - acc: 0.6648    
Epoch 2/5
6250/6250 [==============================] - 17s - loss: 0.3480 - acc: 0.8554    - E
Epoch 3/5
6250/6250 [==============================] - 17s - loss: 0.2220 - acc: 0.9187    
Epoch 4/5
6250/6250 [==============================] - 17s - loss: 0.1508 - acc: 0.9490    - ETA: 1s - loss
Epoch 5/5
6250/6250 [==============================] - 17s - loss: 0.1063 - acc: 0.9672    
Training Model: 1
Epoch 1/5
6250/6250 [==============================] - 18s - loss: 0.5634 - acc

In [8]:
model_store = []

import numpy as np
clients = 4
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
x_train, y_train = unison_shuffled_copies(x_train, y_train)

X_train_split = np.split(x_train, clients)
y_train_split = np.split(y_train, clients)

for i in range(clients):
  print('Training Model:',i)
  model = create_dp_model()
  model.fit(X_train_split[i], y_train_split[i], epochs=1, batch_size=32, verbose=1)
  model_store.append(model)
weights = [model.get_weights() for model in model_store]
epochs = 15
def avg(weights):
  new_weights = []
  for weights_list_tuple in zip(*weights):
      new_weights.append(
          np.array([np.array(weights_).mean(axis=0)\
              for weights_ in zip(*weights_list_tuple)]))
  return np.array(new_weights)

for j in range(epochs):
  weights = [model.get_weights() for model in model_store]
  new_weights = avg(weights)
  global_model = create_dp_model()
  global_model.set_weights(new_weights)
  scores = global_model.evaluate(x_test, y_test, batch_size=32, verbose=1)
  print(j,':',scores)
  for i, model in enumerate(model_store):
    model.set_weights(new_weights)
    model.fit(X_train_split[i], y_train_split[i], epochs=1, batch_size=32, verbose=1)
    #print(model.evaluate(X_test, y_test))

global_model = create_dp_model()
global_model.set_weights(new_weights)
scores = global_model.evaluate(x_test, y_test, batch_size=32, verbose=1)

#scores = global_dp_model.evaluate(X_test, y_test)
print(scores)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Training Model: 0
Epoch 1/1
6250/6250 [==============================] - 29s - loss: 0.6930 - acc: 0.5224    
Training Model: 1
Epoch 1/1
6250/6250 [==============================] - 29s - loss: 0.6931 - acc: 0.5043    
Training Model: 2
Epoch 1/1
6250/6250 [==============================] - 27s - loss: 0.6931 - acc: 0.5064    
Training Model: 3
Epoch 1/1
24992/25000 [============================>.] - ETA: 0s0 : [0.6931323050308228, 0.51176]
Epoch 1/1
6250/6250 [==============================] - 33s - loss: 0.6931 - acc: 0.5234    - ETA: 1s - loss: 0.6931 - acc:
Epoch 1/1
6250/6250 [==============================] - 32s - loss: 0.6931 - acc: 0.5283    
Epoch 1/1
6250/6250 [==============================] - 32s - loss: 0.6931 - acc: 0.5182    
Epoch 1/1
25000/25000 [==============================] - 34s    
1 : [0.6930936739921569, 0.5354]
Epoch 1/1
6250/6250 [==============================] - 34s - loss: 0.6931 - acc: 0.5437    
Epoch 1/1
6250/6250 [==============================] - 34

In [7]:
model_store = []

import numpy as np
clients = 4
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
x_train, y_train = unison_shuffled_copies(x_train, y_train)

X_train_split = np.split(x_train, clients)
y_train_split = np.split(y_train, clients)

for i in range(clients):
  print('Training Model:',i)
  model = create_dp_model()
  model.fit(X_train_split[i], y_train_split[i], epochs=1, batch_size=32, verbose=1)
  model_store.append(model)
weights = [model.get_weights() for model in model_store]
epochs = 20
def avg(weights):
  new_weights = []
  for weights_list_tuple in zip(*weights):
      new_weights.append(
          np.array([np.array(weights_).mean(axis=0)\
              for weights_ in zip(*weights_list_tuple)]))
  return np.array(new_weights)

for j in range(epochs):
  weights = [model.get_weights() for model in model_store]
  new_weights = avg(weights)
  global_model = create_dp_model()
  global_model.set_weights(new_weights)
  scores = global_model.evaluate(x_test, y_test, batch_size=32, verbose=1)
  print(j,':',scores)
  for i, model in enumerate(model_store):
    model.set_weights(new_weights)
    model.fit(X_train_split[i], y_train_split[i], epochs=1, batch_size=32, verbose=1)
    #print(model.evaluate(X_test, y_test))

global_model = create_dp_model()
global_model.set_weights(new_weights)
scores = global_model.evaluate(x_test, y_test, batch_size=32, verbose=1)

#scores = global_dp_model.evaluate(X_test, y_test)
print(scores)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Training Model: 0
Epoch 1/1
6250/6250 [==============================] - 22s - loss: 0.6929 - acc: 0.5197    
Training Model: 1
Epoch 1/1
6250/6250 [==============================] - 22s - loss: 0.6932 - acc: 0.5010    
Training Model: 2
Epoch 1/1
6250/6250 [==============================] - 22s - loss: 0.6930 - acc: 0.5099    
Training Model: 3
Epoch 1/1
24992/25000 [============================>.] - ETA: 0s0 : [0.6931360688781738, 0.50488]
Epoch 1/1
6250/6250 [==============================] - 20s - loss: 0.6931 - acc: 0.5250    
Epoch 1/1
6250/6250 [==============================] - 20s - loss: 0.6931 - acc: 0.5197    
Epoch 1/1
6250/6250 [==============================] - 20s - loss: 0.6931 - acc: 0.5106    
Epoch 1/1
24960/25000 [============================>.] - ETA: 0s1 : [0.693082240524292, 0.54096]
Epoch 1/1
6250/6250 [==============================] - 20s - loss: 0.6930 - acc: 0.5603    
Epoch 1/1
6250/6250 [==============================] - 20s - loss: 0.6930 - acc: 0.5472  

In [9]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=25000, batch_size=32, noise_multiplier=1.3, epochs=15, delta=1e-5)

NameError: name 'compute_dp_sgd_privacy' is not defined

In [10]:
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent

In [11]:
compute_dp_sgd_privacy(n=25000, batch_size=32, noise_multiplier=1.3, epochs=15, delta=1e-5)

NameError: name 'compute_dp_sgd_privacy' is not defined